In [2]:
from pathlib import Path
import numpy as np
import cv2
from matplotlib import pyplot as plt

In [3]:
image_root_path = Path("C:\\Users\\AMET\\DocGaid\\diploma\\data\\full_teeth\\images")
labels_path = Path("C:\\Users\\AMET\\DocGaid\\diploma\\data\\full_teeth\\labels")
single_tooth_output_path = Path("C:\\Users\\AMET\\DocGaid\\yolov8\\data\\new_single_tooth")

txt_file_paths = [path for path in labels_path.glob("*.txt") if 117 <= int(path.stem) <= 210]
image_file_paths = [path for path in image_root_path.glob("*.txt") if 117 <= int(path.stem) <= 210]

In [4]:
def get_class_file_data(file_path):
    with open(file_path, 'r') as file:
        lines = file.readlines()
    bboxes = []
    classes = []
    for line in lines:
        elements = line.split()
        class_id = int(elements[0])
        classes.append(class_id)
        bbox = [float(e) for e in elements[1:]]
        bboxes.append(bbox)
    return np.array(classes), np.array(bboxes)

def xcycwh2xyxy(boxes:np.ndarray):
    # assert boxes.ndim==2 and boxes.shape[1]==4, "The boxes should be of shape n * 4"
    
    boxes[:, 0] -= boxes[:, 2]/2
    boxes[:, 1] -= boxes[:, 3]/2
    boxes[:, 2] += boxes[:, 0]
    boxes[:, 3] += boxes[:, 1]
    
    return boxes   

def rectangles_on_image(image:np.ndarray,
                        boxes:np.ndarray,
                        classes:list = None,
                        box_color:tuple=(255, 0, 0),
                        label_color:tuple=(0, 0, 0),
                        
                        thickness:int=6):
    for box in boxes:
        x1,y1,x2,y2 = box
        cv2.rectangle(image, (x1,y1), (x2,y2), box_color, thickness)
    
    if classes is not None:
        for i, tooth_class in enumerate(classes):
            cv2.putText(image, str(tooth_class), (boxes[i][:2]-10), cv2.FONT_HERSHEY_SIMPLEX, 1.8, label_color, 10)
        
    return image


def denormalize_boxes(center_boxes: np.ndarray, image_shape: tuple[int, int]):
    height, width = image_shape
    # Initialize the denormalized boxes array
    denormalized_boxes = np.zeros_like(center_boxes)
    # Calculate the top-left x,y (x1, y1)
    denormalized_boxes[:, 0] = (center_boxes[:, 0] - center_boxes[:, 2] / 2) * width
    denormalized_boxes[:, 1] = (center_boxes[:, 1] - center_boxes[:, 3] / 2) * height
    # Calculate the bottom-right x,y (x2, y2)
    denormalized_boxes[:, 2] = (center_boxes[:, 0] + center_boxes[:, 2] / 2) * width
    denormalized_boxes[:, 3] = (center_boxes[:, 1] + center_boxes[:, 3] / 2) * height
    
    # Convert to integer coordinates
    denormalized_boxes = denormalized_boxes.astype(int)
    
    return denormalized_boxes

                             

In [ ]:
classes_list = []
target_list = []
for txt_file_path in txt_file_paths:

    target_classes, target_boxes = get_class_file_data(txt_file_path)
    classes_list.append(target_classes)
    target_list.append(target_boxes)
    image_path = (image_root_path/txt_file_path.stem).with_suffix(".jpg")
    # print(target_classes)
    # print(target_boxes)
    image = cv2.imread(str(image_path))
    img_height, img_width, img_channel = image.shape
    img_shape = (img_height, img_width)
    target_boxes = denormalize_boxes(target_boxes, img_shape)
    
    tooth_number_count = 0
    
    for i, box in enumerate(target_boxes):
        x1, y1, x2, y2 = box.astype(int)
        cropped_roi = image[y1: y2, x1: x2]

        cropped_image_path = f"C:\\Users\\AMET\\DocGaid\\yolov8\\data\\new_single_tooth\\{txt_file_path.stem}_{tooth_number_count}.jpg"
        cv2.imwrite(cropped_image_path, cropped_roi)

        tooth_number_count += 1
        plt.imshow(cv2.cvtColor(cropped_roi, cv2.COLOR_BGR2RGB))
        plt.axis('off')
        plt.show()

    plt.imshow(rectangles_on_image(image.copy(), denormalize_boxes(target_boxes, img_shape), target_classes))
    plt.show()
